<a href="https://colab.research.google.com/github/CUBigDataClass/Collatz/blob/main/Hotels_Flights_CarRentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U airportsdata
!pip install geopy

import requests
import json
import airportsdata
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim

# Pure data
airports = airportsdata.load()

# Processed Data
airports_df = pd.DataFrame(columns=["code", "lat", "long"])

for key in airports.keys():
  if ("international" in airports[key]["name"].lower()) or ("intl" in airports[key]["name"].lower()):
    if airports[key]["iata"] is not "":
      # print(airports[key])
      temp_df = pd.DataFrame([[airports[key]["iata"], airports[key]["lat"], airports[key]["lon"]]], columns=["code", "lat", "long"])
      airports_df = airports_df.append(temp_df, ignore_index=True)

In [ ]:
airports_df.head()

,code,lat,long
0,HIR,-9.428000,160.054993
1,INU,-0.547458,166.919006
2,POM,-9.443380,147.220001
3,WWK,-3.583830,143.669006
4,KEF,63.985001,-22.605600


In [ ]:
starting_loc=input("Where are you leaving from? ")
destination=input("Where would you like to visit? ")
start_date=input("When will you be leaving? (YYYY-MM-DD) ")
end_date=input("When will you be coming back? (YYYY-MM-DD) ")
adult_count=int(input("How many adults will be coming on this trip? "))
child_count=int(input("How many children will be coming on this trip? "))

def getBestAirports(location, airports_df, nbest=3):
  geocoder = Nominatim(user_agent="GetLoc")
  getLocation = geocoder.geocode(location)

  lat = getLocation.latitude
  long = getLocation.longitude

  location = geocoder.reverse((lat, long))

  # print(location.raw["latitude"])

  city = location.raw["address"]["city"]
  if "state" in location.raw["address"].keys():
    state = location.raw["address"]["state"]
  elif "region" in location.raw["address"].keys():
    state = location.raw["address"]["region"]

  df = airports_df.copy()

  df["dist"] = np.sqrt((airports_df["lat"] - lat)**2 + (airports_df["long"] - long)**2)
  best_airports = df.nsmallest(nbest, columns=["dist"])
  best_airports.reset_index(inplace=True)

  return best_airports, city, state

home_best_airports, home_city, home_state = getBestAirports(starting_loc, airports_df)
dest_best_airports, dest_city, dest_state = getBestAirports(destination, airports_df)
# print(location.raw["address"]["country_code"].upper())

Where are you leaving from? Boulder
Where would you like to visit? Shanghai
When will you be leaving? (YYYY-MM-DD) 2022-06-17
When will you be coming back? (YYYY-MM-DD) 2022-06-25
How many adults will be coming on this trip? 1
How many children will be coming on this trip? 0


In [ ]:
home_airport_code = home_best_airports.loc[0]["code"]
dest_airport_code = dest_best_airports.loc[0]["code"]

print(dest_best_airports)

   index code        lat        long      dist
0    834  SHA  31.197901  121.335999  0.137572
1    833  PVG  31.143400  121.805000  0.347356
2    836  WUX  31.494400  120.429001  1.072725


In [ ]:
key = "Insert Rapid API Key here"

url = "https://priceline-com-provider.p.rapidapi.com/v1/hotels/locations"

querystring = {"search_type":"HOTEL","name":f"{dest_city}, {dest_state}"}

headers = {
    'x-rapidapi-host': "priceline-com-provider.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

dictionary = json.loads(response.text)
city_id = dictionary[0]["cityID"]

print(city_id)


3000040015


In [ ]:
location_id = city_id
checkin = start_date
checkout = end_date
num_rooms = str(math.ceil(adult_count/2))

url = "https://priceline-com-provider.p.rapidapi.com/v1/hotels/search"

querystring = {"date_checkin":checkin,"location_id":location_id,"date_checkout":checkout,"sort_order":"PRICE","amenities_ids":"FINTRNT,FBRKFST","rooms_number":num_rooms,"star_rating_ids":"3.0,3.5,4.0,4.5,5.0"}

headers = {
    'x-rapidapi-host': "priceline-com-provider.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

hotels_for_city = json.loads(response.text)

with open("drive/MyDrive/hotels.json", "w") as file:
	json.dump(hotels_for_city, file)

In [ ]:
url = "https://priceline-com-provider.p.rapidapi.com/v2/flight/roundTrip"

# UNCOMMENT FOR A WORKING EXAMPLE
# home_airport_code = "DEN"
# dest_airport_code = "JFK"

print(dest_airport_code)

querystring = {"departure_date":f"{checkin},{checkout}","adults":f"{adult_count}","sid":"iSiX639","origin_airport_code":f"{home_airport_code},{dest_airport_code}","destination_airport_code":f"{dest_airport_code},{home_airport_code}"}

headers = {
    'x-rapidapi-host': "priceline-com-provider.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(json.loads(response.text))

SHA
{'getAirFlightRoundTrip': {'results': {'status': 'Success', 'status_code': 100, 'result_code': 0, 'result_message': 'Success', 'air_search_rsp': {'client_session_id': 'iSiX639', 'point_of_sale': {'request_source_country': None, 'country': 'US', 'currency': 'USD'}, 'image_path': 'https://www.priceline.com/sam/air/carrier_logos/', 'airport': [{'code': 'DEN', 'name': 'Denver Intl Airport', 'city': 'Denver', 'state': 'CO', 'country': 'United States', 'latitude': 39.852709694, 'longitude': -104.67383148, 'iso_country_code': 'US', 'city_id': '1082000019', 'city_gis_id': '3000002558', 'gmt_time_zone_offset': '-07:00'}, {'code': 'SHA', 'name': 'Hongqiao Intl Airport', 'city': 'Shanghai', 'state': None, 'country': 'China', 'latitude': 31.198182198, 'longitude': 121.332407486, 'iso_country_code': 'CN', 'city_id': '2000000944', 'city_gis_id': '3000040015', 'gmt_time_zone_offset': '+08:00'}, {'code': 'JFK', 'name': 'John F Kennedy Intl Airport', 'city': 'New York City', 'state': 'NY', 'country

In [ ]:
dict = json.loads(response.text)

if "error" not in dict["getAirFlightRoundTrip"].keys():
  min_cost = dict["getAirFlightRoundTrip"]["results"]["air_search_rsp"]["total_trip_summary"]["minTotalFareWithTaxesAndFees"]
  print(min_cost)


13284.77


In [ ]:
url = "https://priceline-com-provider.p.rapidapi.com/v2/cars/resultsRequest"

querystring = {"pickup_date":checkin,"dropoff_date":checkout,"dropoff_time":"10:00","pickup_time":"10:00","pickup_airport_code":dest_airport_code,"currency":"USD"}

headers = {
    'x-rapidapi-host': "priceline-com-provider.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

rentals = json.loads(response.text)

In [ ]:
rentals

{'getCarResultsRequest': {'results': {'car_companies': {'company_0': {'code': 'HZ',
     'name': 'Hertz'}},
   'rc_inventory': 'PCLN_RC',
   'rc_inventory_count': 7,
   'results_list': {'result_0': {'car': {'air_conditioning': 'true',
      'allow_cancellation': True,
      'bags': 2,
      'campaign_rate': 'false',
      'description': 'Compact SUV',
      'discount': 0,
      'doors': '5',
      'example': 'Compact SUV  Borgward BX5 or similar',
      'free_cancellation': True,
      'imageURL': 'https://s1.pclncdn.com/rc-static/vehicles/partner/hz/size134x72/zecncfar999.jpg',
      'images': {'SIZE134X72': 'https://s1.pclncdn.com/rc-static/vehicles/partner/hz/size134x72/zecncfar999.jpg',
       'SIZE268X144': 'https://s1.pclncdn.com/rc-static/vehicles/partner/hz/size268x144/zecncfar999.jpg',
       'SIZE67X36': 'https://s1.pclncdn.com/rc-static/vehicles/partner/hz/size67x36/zecncfar999.jpg'},
      'inclusions': [],
      'mileage': True,
      'partner_discounts': None,
      'pass